# CDS Strategy Testing and P&L Calculations

This notebook implements and tests various CDS trading strategies.

In [1]:
# Setup
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Import our modules
from src.data import BloombergCDSConnector, CDSDataManager
from src.models import (
    Region, Market, Tenor, Side,
    Position, Strategy,
    CDSDatabase
)

print('Modules imported successfully')

Modules imported successfully


## 1. Initialize Components

In [2]:
# Initialize
connector = BloombergCDSConnector()
db = CDSDatabase('../data/cds_monitor.db')
manager = CDSDataManager(connector, db)

# Current on-the-run series
current_series = 43

print(f'Working with series {current_series}')

Working with series 43


## 2. 5s10s Steepener Strategy

In [3]:
# Calculate 5s10s metrics
print('Analyzing 5s10s Steepener...\n')

metrics = manager.calculate_strategy_metrics('5s10s', series=current_series)

if metrics:
    print('Current Market:')
    print(f'5Y Spread: {metrics["spread_5y"]:.2f} bps')
    print(f'10Y Spread: {metrics["spread_10y"]:.2f} bps')
    print(f'5s10s Slope: {metrics["slope"]:.2f} bps')
    print(f'\nRisk Metrics:')
    print(f'5Y DV01: ${metrics["dv01_5y"]:,.0f}')
    print(f'10Y DV01: ${metrics["dv01_10y"]:,.0f}')
    print(f'DV01 Ratio (10Y/5Y): {metrics["dv01_ratio"]:.3f}')

Analyzing 5s10s Steepener...

Current Market:
5Y Spread: 50.34 bps
10Y Spread: 91.69 bps
5s10s Slope: 41.35 bps

Risk Metrics:
5Y DV01: $4,587
10Y DV01: $8,295
DV01 Ratio (10Y/5Y): 1.808


In [4]:
# Debug the metrics first
print("DEBUG - Checking metrics:")
print(f"metrics = {metrics}")
if metrics:
    print(f"dv01_ratio = {metrics.get('dv01_ratio', 'KEY MISSING')}")
    print(f"np.isnan(dv01_ratio) = {np.isnan(metrics.get('dv01_ratio', float('nan')))}")
else:
    print("metrics is None - check calculate_strategy_metrics")



# Create DV01-neutral 5s10s steepener position
if metrics and not np.isnan(metrics['dv01_ratio']):
    # Calculate notionals for DV01 neutrality
    notional_5y = 10_000_000  # $10mm 5Y
    notional_10y = notional_5y / metrics['dv01_ratio']  # Scaled for DV01 neutrality
    
    print(f'Creating DV01-neutral steepener:')
    print(f'Sell 5Y: ${notional_5y:,.0f}')
    print(f'Buy 10Y: ${notional_10y:,.0f}')
    
    # Create strategy
    steepener = Strategy(
        name=f'EU_IG_5s10s_S{current_series}',
        strategy_type='5s10s',
        positions=[
            Position(
                index_id=f'EU_IG_S{current_series}_5Y',
                side=Side.SELL,
                notional=notional_5y,
                entry_date=datetime.now(),
                entry_spread=metrics['spread_5y'],
                entry_dv01=metrics['dv01_5y']
            ),
            Position(
                index_id=f'EU_IG_S{current_series}_10Y',
                side=Side.BUY,
                notional=notional_10y,
                entry_date=datetime.now(),
                entry_spread=metrics['spread_10y'],
                entry_dv01=metrics['dv01_10y']
            )
        ],
        creation_date=datetime.now(),
        target_dv01=0,
        stop_loss=50000,  # $50k stop loss
        take_profit=100000  # $100k take profit
    )
    
    print(f'\nStrategy created:')
    print(f'Net DV01: ${steepener.net_dv01:,.0f}')
    print(f'Is DV01 neutral: {steepener.is_dv01_neutral(tolerance=1000)}')
    
    # Save to database
    db.save_strategy(steepener)
    print('Strategy saved to database')

DEBUG - Checking metrics:
metrics = {'series': 43, 'spread_5y': 50.3357, 'spread_10y': 91.68678, 'slope': 41.351079999999996, 'dv01_5y': 4587.283931699821, 'dv01_10y': 8294.855400804836, 'dv01_ratio': 1.8082280330380973, 'dv01_weighted_slope': 74.77218205240099, 'timestamp': datetime.datetime(2025, 9, 16, 14, 1, 58, 169015)}
dv01_ratio = 1.8082280330380973
np.isnan(dv01_ratio) = False
Creating DV01-neutral steepener:
Sell 5Y: $10,000,000
Buy 10Y: $5,530,276

Strategy created:
Net DV01: $0
Is DV01 neutral: True
Strategy saved to database


In [6]:
# Check available methods
print("Available database methods:")
print([method for method in dir(db) if not method.startswith('_')])

Available database methods:
['close', 'conn', 'db_path', 'get_historical_spreads', 'get_latest_spread', 'get_open_positions', 'get_pnl_history', 'save_curve', 'save_index_definition', 'save_pnl_history', 'save_position', 'save_spread_data', 'save_strategy', 'update_position_exit']


In [7]:
# Check what tables exist
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(query, db.conn)
print("Available tables:")
print(tables)

# Check the strategies table structure and data
if 'strategies' in tables['name'].values:
    # Get table schema
    schema = pd.read_sql_query("PRAGMA table_info(strategies)", db.conn)
    print("\nStrategies table schema:")
    print(schema)
    
    # Get saved strategies
    strategies_df = pd.read_sql_query("SELECT * FROM strategies ORDER BY creation_date DESC", db.conn)
    print(f"\nSaved strategies ({len(strategies_df)} total):")
    print(strategies_df)
    
    # Check positions table if it exists
    if 'positions' in tables['name'].values:
        positions_df = pd.read_sql_query("SELECT * FROM positions ORDER BY entry_date DESC", db.conn)
        print(f"\nSaved positions ({len(positions_df)} total):")
        print(positions_df.head())
else:
    print("No strategies table found - data might not be saving correctly")

Available tables:
                name
0  index_definitions
1        spread_data
2    sqlite_sequence
3             curves
4          positions
5         strategies
6        pnl_history

Strategies table schema:
   cid           name       type  notnull         dflt_value  pk
0    0  strategy_name       TEXT        0               None   1
1    1  strategy_type       TEXT        1               None   0
2    2  creation_date  TIMESTAMP        1               None   0
3    3    target_dv01       REAL        0               None   0
4    4      stop_loss       REAL        0               None   0
5    5    take_profit       REAL        0               None   0
6    6         status       TEXT        0           'ACTIVE'   0
7    7       metadata       TEXT        0               None   0
8    8     created_at  TIMESTAMP        0  CURRENT_TIMESTAMP   0
9    9     updated_at  TIMESTAMP        0  CURRENT_TIMESTAMP   0

Saved strategies (1 total):
     strategy_name strategy_type            

In [5]:
# Check what strategies are saved
saved_strategies = db.get_all_strategies()
print(f"Number of saved strategies: {len(saved_strategies)}")

if saved_strategies:
    for strategy in saved_strategies[-3:]:  # Show last 3
        print(f"\nStrategy: {strategy.name}")
        print(f"Type: {strategy.strategy_type}")
        print(f"Created: {strategy.creation_date}")
        print(f"Positions: {len(strategy.positions)}")
        print(f"Net DV01: ${strategy.net_dv01:,.0f}")

AttributeError: 'CDSDatabase' object has no attribute 'get_all_strategies'

In [8]:
# Get the most recent strategy positions
recent_positions = pd.read_sql_query("""
    SELECT * FROM positions 
    WHERE strategy_name = 'EU_IG_5s10s_S43' 
    AND entry_date = (
        SELECT MAX(entry_date) FROM positions 
        WHERE strategy_name = 'EU_IG_5s10s_S43'
    )
""", db.conn)

print("Most recent strategy positions:")
print(recent_positions[['index_id', 'side', 'notional', 'entry_spread', 'entry_dv01']])

# Calculate current P&L
total_pnl = 0
print(f"\nP&L Analysis (vs current spreads):")
print(f"Current 5Y spread: {metrics['spread_5y']:.2f} bps")
print(f"Current 10Y spread: {metrics['spread_10y']:.2f} bps")
print("-" * 50)

for _, pos in recent_positions.iterrows():
    # Get current spread
    if '5Y' in pos['index_id']:
        current_spread = metrics['spread_5y']
    elif '10Y' in pos['index_id']:
        current_spread = metrics['spread_10y']
    
    # Calculate P&L: (current - entry) * DV01 * side
    spread_change = current_spread - pos['entry_spread']
    position_pnl = spread_change * pos['entry_dv01'] * pos['side']  # side is -1 for SELL, +1 for BUY
    total_pnl += position_pnl
    
    side_text = "SELL" if pos['side'] == -1 else "BUY"
    print(f"{pos['index_id']} ({side_text}):")
    print(f"  Entry: {pos['entry_spread']:.2f} bps")
    print(f"  Current: {current_spread:.2f} bps") 
    print(f"  Change: {spread_change:+.2f} bps")
    print(f"  DV01: ${pos['entry_dv01']:,.0f}")
    print(f"  P&L: ${position_pnl:,.0f}")
    print()

print(f"🎯 **Total Strategy P&L: ${total_pnl:,.0f}**")

# Check if strategy hit profit targets
strategy_info = pd.read_sql_query("SELECT * FROM strategies WHERE strategy_name = 'EU_IG_5s10s_S43'", db.conn)
stop_loss = strategy_info['stop_loss'].iloc[0]
take_profit = strategy_info['take_profit'].iloc[0]

print(f"\nStrategy Targets:")
print(f"Stop Loss: ${stop_loss:,.0f}")
print(f"Take Profit: ${take_profit:,.0f}")

if total_pnl <= -stop_loss:
    print("🔴 STOP LOSS HIT!")
elif total_pnl >= take_profit:
    print("🟢 TAKE PROFIT HIT!")
else:
    print(f"Position within targets ({total_pnl/take_profit*100:.1f}% to take profit)")

Most recent strategy positions:
        index_id  side      notional  entry_spread   entry_dv01
0   EU_IG_S43_5Y    -1  1.000000e+07      50.33570  4587.283932
1  EU_IG_S43_10Y     1  5.530276e+06      91.68678  8294.855401

P&L Analysis (vs current spreads):
Current 5Y spread: 50.34 bps
Current 10Y spread: 91.69 bps
--------------------------------------------------
EU_IG_S43_5Y (SELL):
  Entry: 50.34 bps
  Current: 50.34 bps
  Change: +0.00 bps
  DV01: $4,587
  P&L: $-0

EU_IG_S43_10Y (BUY):
  Entry: 91.69 bps
  Current: 91.69 bps
  Change: +0.00 bps
  DV01: $8,295
  P&L: $0

🎯 **Total Strategy P&L: $0**

Strategy Targets:
Stop Loss: $50,000
Take Profit: $100,000
Position within targets (0.0% to take profit)


## 3. Compression Trade (Main vs Crossover)

In [9]:
# Analyze compression opportunity
print('Analyzing EU Compression Trade...\n')

compression_metrics = manager.calculate_strategy_metrics('compression', series=current_series)

if compression_metrics:
    print('Current Market:')
    print(f'Main (IG) Spread: {compression_metrics["main_spread"]["PX_LAST"].iloc[0]:.2f} bps')
    print(f'Crossover Spread: {compression_metrics["xo_spread"]["PX_LAST"].iloc[0]:.2f} bps')
    print(f'XO-Main Basis: {compression_metrics["basis"]:.2f} bps')
    print(f'XO/Main Ratio: {compression_metrics["ratio"]:.2f}x')

Analyzing EU Compression Trade...



## 4. P&L Simulation

In [12]:
# Simulate P&L for different spread moves
def simulate_pnl(strategy, spread_changes):
    '''Simulate P&L for given spread changes'''
    total_pnl = 0
    
    for position in strategy.positions:
        if position.index_id in spread_changes:
            spread_change = spread_changes[position.index_id]  # in bps
            
            # Map side to multiplier properly
            if hasattr(position.side, 'value'):
                side_str = position.side.value
            else:
                side_str = str(position.side)
                
            if side_str == 'SELL' or side_str == -1:
                side_multiplier = -1
            elif side_str == 'BUY' or side_str == 1:
                side_multiplier = 1
            else:
                print(f"Unknown side: {side_str}")
                continue
            
            pnl = spread_change * position.entry_dv01 * side_multiplier
            print(f"{position.index_id}: {spread_change}bps × ${position.entry_dv01} × {side_multiplier} = ${pnl:,.0f}")
            
            total_pnl += pnl
    
    return total_pnl

# Simulate scenarios for the steepener
if 'steepener' in locals():
    print('P&L Simulation for 5s10s Steepener:\n')
    
    scenarios = [
        {'name': 'Parallel shift +5bps', 
         'changes': {f'EU_IG_S{current_series}_5Y': 5, f'EU_IG_S{current_series}_10Y': 5}},
        {'name': 'Steepening +5bps', 
         'changes': {f'EU_IG_S{current_series}_5Y': 0, f'EU_IG_S{current_series}_10Y': 5}},
        {'name': 'Flattening -5bps', 
         'changes': {f'EU_IG_S{current_series}_5Y': 5, f'EU_IG_S{current_series}_10Y': 0}},
    ]
    
    for scenario in scenarios:
        pnl = simulate_pnl(steepener, scenario['changes'])
        print(f'{scenario["name"]}: ${pnl:,.0f}')

P&L Simulation for 5s10s Steepener:

EU_IG_S43_5Y: 5bps × $4587.283931699821 × -1 = $-22,936
EU_IG_S43_10Y: 5bps × $8294.855400804836 × 1 = $41,474
Parallel shift +5bps: $18,538
EU_IG_S43_5Y: 0bps × $4587.283931699821 × -1 = $-0
EU_IG_S43_10Y: 5bps × $8294.855400804836 × 1 = $41,474
Steepening +5bps: $41,474
EU_IG_S43_5Y: 5bps × $4587.283931699821 × -1 = $-22,936
EU_IG_S43_10Y: 0bps × $8294.855400804836 × 1 = $0
Flattening -5bps: $-22,936
